## POS tagging using modified Viterbi Algorithm:

In this Project the following are the steps followed:

1. Plain vanilla Viterbi Algorithm is developed.
2. Viterbi Modification- I:
    * Transition probability is considered in case of unknown words(as the Emission is 0).
    * Multiplying the transition probability weighted by tag occurrence in training set.
3. Viterbi Modification- II:
     * Rule based tagger in case of an unknown word.
     * Multipying the transition probability to rule based tagger and also returns default tag ('NN').
4. Tested the modified algorithm on validation test data.
5. Comparing the final and vanilla algorithms performance on actual test data.

### Data Preparation

In [53]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import time

In [54]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [55]:
# sample check of the tagged data
print(nltk_data[1:5])

[[('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('conglomerate', 'NOUN'), ('.', '.')], [('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('cigarette', 'NOUN'), ('filters', 'NOUN'), ('has', 'VERB'), ('caused', 'VERB'), ('a', 'DET

In [56]:
# split data into training and validation set in the ratio 95:5
train_set,test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state = 101)

In [57]:
# Creating List of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

95547
5129


In [58]:
# Sample check of the tagged words.
train_tagged_words[1:5]

[('confirmed', 'VERB'), ('the', 'DET'), ('filing', 'NOUN'), ('but', 'CONJ')]

In [59]:
# Unique tags in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'ADJ', 'DET', 'X', 'ADV', 'PRON', 'VERB', 'NUM', 'CONJ', 'NOUN', 'ADP', 'PRT', '.'}


In [60]:
# Total nuber of words in training set
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

12100


### POS Tagging algorithm using Hidden Markov Model (HMM)

**Goal**: Given a sequence of words, assign the most probable tag to the word. 
In other words, to every **word w**, assign **the tag t** that maximises the likelihood **P(t|w)**. 

P(t|w) = P(w|t)* P(t) / P(w), ignore P(w).

* **P(w|t): is the emission probability** of a given word for a given tag. This can be computed based on the fraction of given word for given tag to the total count of that tag, i.e: P(w|t) = count(w, t) / count(t). 

* **P(t): is the probability of tag (also transition probability)**, and in a tagging task, we assume that a tag will depend only on the previous tag (Markov order 1 assumption). p(t2|t1)= count(t2|t1) / count(t1)

### Build the Vanilla Viterbi based POS tagger

In [61]:
# Emission probability for a given word and tag
def word_given_tag(word,tag,train_bag= train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(taglist)    
    w_in_tag = [pair[0] for pair in taglist if pair[0]==word]    
    word_count_given_tag = len(w_in_tag)    
    
    return (word_count_given_tag,tag_count)

In [62]:
# Transition probabilities of a previous and Current tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    
    t1_tags = [tag for tag in tags if tag==t1]
    
    count_of_t1 = len(t1_tags)
    
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    
    count_t2_given_t1 = len(t2_given_t1)
    
    return(count_t2_given_t1,count_of_t1)

In [63]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [64]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [65]:
tags_df

,ADJ,DET,X,ADV,PRON,VERB,NUM,CONJ,NOUN,ADP,PRT,.
ADJ,0.066403,0.004943,0.021091,0.004778,0.000330,0.011699,0.021256,0.016971,0.699621,0.078267,0.010710,0.063931
DET,0.204323,0.005676,0.045405,0.012438,0.003744,0.039850,0.022220,0.000483,0.638087,0.009540,0.000242,0.017993
X,0.017187,0.054742,0.076384,0.024984,0.055538,0.203851,0.002864,0.010662,0.062381,0.142584,0.185232,0.163590
ADV,0.129182,0.069891,0.023186,0.080490,0.014906,0.343491,0.030474,0.006956,0.031467,0.118582,0.014243,0.137131
PRON,0.073124,0.009954,0.089969,0.034074,0.007657,0.485452,0.006508,0.005360,0.210949,0.022971,0.013017,0.040965
VERB,0.064988,0.134392,0.217506,0.081952,0.035786,0.169249,0.022851,0.005577,0.110070,0.092022,0.030674,0.034934
NUM,0.034247,0.003276,0.210542,0.002978,0.001489,0.018761,0.184932,0.013699,0.350208,0.036033,0.026504,0.117332
CONJ,0.118085,0.121339,0.008833,0.055323,0.058113,0.156671,0.039981,0.000465,0.349140,0.052534,0.004649,0.034868
NOUN,0.012248,0.012942,0.029175,0.017074,0.004607,0.147667,0.009542,0.042666,0.263564,0.176514,0.043397,0.240604
ADP,0.107024,0.324709,0.034427,0.014006,0.070031,0.008340,0.062226,0.000962,0.320967,0.016893,0.001390,0.039025


### Viterbi Algorithm

The steps to follow:

1. Read the sequence of words.
2. Iterate through the words
3. Read the current word and Calculate the product of emission probabilties and transition probabilties of the word for all possible tags.
4. Assign the tag which has maximum probability..
5. Repeat steps from step 2.

#### The function to implement the Vanilla Viterbi algorithm

In [66]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # assigning state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Testing Vanilla Viterbi Algorithm on sampled test data

In [67]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [68]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  43.04786396026611
Vanilla Viterbi Algorithm Accuracy:  89.38053097345133


In [69]:
# Incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('Contra', 'ADJ'), ('Contra', 'NOUN')),
 (('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'ADJ'), ('Honduras', 'NOUN')),
 (('Sandinista', 'ADJ'), ('Sandinista', 'NOUN')),
 (('offensive', 'ADJ'), ('offensive', 'NOUN')),
 (('rebel', 'ADJ'), ('rebel', 'NOUN')),
 (('Bucking', 'ADJ'), ('Bucking', 'VERB')),
 (('drew', 'ADJ'), ('drew', 'VERB')),
 (('Eveready', 'ADJ'), ('Eveready', 'NOUN')),
 (('*T*-252', 'ADJ'), ('*T*-252', 'X')),
 (('complaining', 'ADJ'), ('complaining', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

#### Observation
We can see that Most of unknown words(Emission Prob = 0) have been tagged as 'ADJ' instead of 'NOUN' because the first tag in tag list is ADJ.

### Viterbi Modification - I:

**Step 1: Modify algorithm so as to assign only transition probabilities to unknown words as emission probability is zero.**

In [70]:
# Assign only transition probability of tags when emission probability is zero

def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [71]:
# Tagging on test words
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  43.95623445510864
Modified Viterbi_1 Accuracy:  94.69026548672566


**Step 2: Multiplying the Probability weights of each tag based on their occurence**

* Applying weights based on the probability of tag occurance to the transition probabilities of each tag and then use the resulting probability for predicting unknown words. 

* This would be useful as some of the POS tags are more likely to occur as compared to others according to the training data.

In [72]:
# Creating a list containing tuples of POS tags its probability, based on training data
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('ADJ', 0.06351847781719991),
 ('DET', 0.08666938784053921),
 ('X', 0.06576867928872701),
 ('ADV', 0.031597015081582885),
 ('PRON', 0.0273373313657153),
 ('VERB', 0.1351167488251855),
 ('NUM', 0.035145007169246546),
 ('CONJ', 0.02251248076862696),
 ('NOUN', 0.2862674913916711),
 ('ADP', 0.0978889970381069),
 ('PRT', 0.03176447193527793),
 ('.', 0.11641391147812072)]

In [73]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [74]:
# tagging the test Words
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  33.26298952102661
Modified Viterbi_1 Accuracy:  95.57522123893806


**Observation:**
Better accuracy by using weighted transition probabilties.

In [75]:
# check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_1 as compared to vanilla Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

### Viterbi Modification- II

**Step 1: Uing Rule based tagger For handling unknown words.**

Describing some Standard Rules for assigning a tag

In [76]:
# Patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [77]:
# Modified Viterbi Algorithm using Rule based tagger for unknown words.
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [78]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  37.274662017822266


In [79]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Modified Viterbi_2 Accuracy:  97.34513274336283


In [80]:
# Check incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('drew', 'NOUN'), ('drew', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'

**Step 2:Multipying the transition probability to rule based tagger and also returns default tag ('NN').**

Changes:
* Modifying the rule based tagger to output 'NN' instead of 'NOUN' in case word does not satisfy any rules.
* We also observe that any capitalized word can still be defaulted as 'NOUN'..

In [81]:
# Patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [82]:
# modified Viterbi with above rules and multiply with Tag Probability
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        
      
        # getting state for which probability is maximum
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            
            # if unknown word does not satisfy any rule, find the tag with maximum transition probability
            if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                 
                
        else:
             if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [83]:
# tagging the test Words
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  38.76617503166199


In [84]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Modified Viterbi Algorithm Accuracy:  98.23008849557522


**Observation**: Accuracy is much better comapared to previous.

In [85]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')), (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:
* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'
* drew: correctly tagged as VERB

#### Evaluating vanilla Viterbi with modified Viterbi on splitted validation test data

In [86]:
# List containing all the words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
# List containing the tuples of word and tag( For accuracy check )
test_run_base = [tup for sent in test_set for tup in sent]

In [87]:
# tagging the test Words
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  1739.9557704925537
Modified Viterbi Algorithm Accuracy:  91.79177227529733


In [88]:
# tagging the test words
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  1762.0549039840698
Modified Viterbi Algorithm Accuracy:  95.43770715539091


#### Evaluating tagging on sample 'Test_sentences.txt' file

In [89]:
f = open('Test_sentences.txt')

In [90]:
text = f.read()

In [91]:
sample_test_sent = text.splitlines()

In [92]:
f.close()

In [93]:
sample_test_sent

['Android is a mobile operating system developed by Google.',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
 'Show me the cheapest round trips from Dallas to Atlanta',
 'I would like to see flights from Denver to Philadelphia.',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.',
 'NASA invited social media users to experienc

In [94]:
# list of untagged words
sample_test_words = [word for sent in sample_test_sent for word in sent.split()]

In [95]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  63.475900650024414


In [96]:
sample_tagged_seq

[('Android', 'ADJ'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'ADJ'),
 ('Android', 'ADJ'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'ADJ'),
 ('worldwide', 'ADJ'),
 ('on', 'ADP'),
 ('smartphones', 'ADJ'),
 ('since', 'ADP'),
 ('2011', 'ADJ'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'ADJ'),
 ('Google', 'ADJ'),
 ('and', 'CONJ'),
 ('Twitter', 'ADJ'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'ADJ'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'ADJ'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'ADJ'),
 ('firehose.', 'ADJ'),
 ('Twitter', 'ADJ'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADJ'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET'),
 ('users',

We can see that several words have been misclassified by vanilla Viterbi POS tagger, for example:
* Android as ADJ
* Google as ADJ
* OS as ADJ

In [97]:
# tagging the test sentences using Modified algorithm
start = time.time()
sample_tagged_seq = Viterbi_2(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  67.74616432189941


In [98]:
sample_tagged_seq

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

The following are few Words were correctly POS tagged by Modified Algotithm:
* Android as NOUN
* Google as NOUN
* OS as NOUN

#### Compare the accuracy of the vanilla Viterbi with Modified Viterbi algorithm

The accuracy of Vanilla Viterbi Algorithm: **91.79%**

The accuracy of Modified Viterbi Algorithm: **95.44%**

The following Words which were incorrectly tagged and got corrected by the Modified Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'
* drew: correctly tagged as VERB